<a href="https://colab.research.google.com/github/analiacuracaram/UGR_DeepLearning/blob/main/Trabajo_Practico_N1_Aprendizaje_Profundo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###**Alumnas**
      Guadalupe Solé

      Analía Cura Caram

##Readme
Este trabajo se abordó siguiendo la documentación citada en https://www.physionet.org/content/mitdb/1.0.0/

Acorde a como allí se solicita, se cita la fuente de información:

*Goldberger, A., Amaral, L., Glass, L., Hausdorff, J., Ivanov, P. C., Mark, R., ... & Stanley, H. E. (2000). PhysioBank, PhysioToolkit, and PhysioNet: Components of a new research resource for complex physiologic signals. Circulation [Online]. 101 (23), pp. e215–e220.*

#Paso 1 - Dataset

Documentación wfdb https://physionet.org/content/wfdb-python/3.4.1/

In [1]:
!pip install wfdb #Instalamos el paquete wfdb para poder manejar los waveform

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 31.0 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.


##1.1 Importamos el dataset

In [2]:
import wfdb
waveform_db = 'mitdb'

In [7]:
subjects = wfdb.get_record_list(waveform_db)
print(f"La '{waveform_db}' base de datos contiene {len(subjects)} muestras")

ConnectTimeout: HTTPSConnectionPool(host='physionet.org', port=443): Max retries exceeded with url: /content/mitdb/ (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x79980f335a10>, 'Connection to physionet.org timed out. (connect timeout=None)'))

In [ ]:
import wfdb
import matplotlib.pyplot as plt

record1 = '100' #Graficamos el primer registro para probar la importación
record = wfdb.rdrecord(record1, pn_dir='mitdb')

plt.figure(figsize=(15, 4))
for i in range(record.p_signal.shape[1]):
    plt.plot(record.p_signal[:, i], label=f'Señal {i+1}')
plt.xlabel('Muestras')
plt.ylabel('Amplitud')
plt.legend()
plt.grid(True)
plt.show()


###1.2 Clasificación en 3 clases
Normal(N), Contracción Ventricular Prematura (PVC) y Latido Supraventricular (APB)

In [ ]:
import pandas as pd

In [ ]:
data = []
clases = ['N', 'V', 'A'] #definimos las clases de interés. "Atrial premature beat=A" es un tipo de latido supraventricular y coincide con las siglas de la consigna

lista_registros = wfdb.get_record_list('mitdb') #Obtenemos la lista de registros

for nombre_registros in lista_registros: #Recorremos los registros
    annotation = wfdb.rdann(nombre_registros, 'atr', pn_dir='mitdb') #Cargamos las anotaciones

    record = wfdb.rdrecord(nombre_registros, pn_dir='mitdb') #Cargamos señal completa, con sus dos canales (nombre del registro y directorio)
    signal = record.p_signal[:, 0]  ##xtraemos el canal 0 que es el principal para manejarlo como un vector (no como una matriz registro/canal)

    for index, label in zip(annotation.sample, annotation.symbol): #Sample son los indices de cada anotación de latido, symbol son las etiquetas. Zip combina ambos valores
        if label in clases: #Filtramos las clases de interés para no recorrer todo
            window_size = 200 #Longitud de ventana razonable https://www.sciencedirect.com/science/article/abs/pii/S1746809413000062
            if index - window_size//2 >= 0 and index + window_size//2 <= len(signal): #Verificamos que el indice de la muestra este dentro de la ventana
                beat = signal[index - window_size//2 : index + window_size//2]
                data.append({
                    'record': nombre_registros,
                    'sample': index,
                    'label': label,
                    'signal': beat
                })

df = pd.DataFrame(data)

print(f"DataFrame construido con {len(df)} latidos.")
df.head()

#Paso 2 - Preprocesamiento

1. Nos enfocamos solo en 3 clases: 'N' → Latido normal (Normal beat). 'V' → Contracción ventricular prematura (PVC),'A' → Latido supraventricular prematuro (APB), descartando otros tipos de latidos
2. Recorremos cada registro de la base MIT-BIH; se filtra toda la señal usando un filtro Butterworth pasa-bajos de 40 Hz para reducir el ruido.  https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.butter.html
3. Segmentación: Se corta una ventana de 200 milisegundos (72 muestras) alrededor del pico. Se extrae solo la señal que rodea al evento cardíaco.
4. Normalización: Cada ventana segmentada se normaliza: Media = 0, Desviación estándar = 1; de esta forma quedan los latidos comparables entre si.
Cada latido filtrado, segmentado y normalizado se guarda en un DataFrame (df)



##2.1/2.2 Filtrado y Segmentación

In [ ]:
## 1. Instalamos e importamos librerías
import numpy as np
from scipy.signal import butter, filtfilt
from sklearn.model_selection import train_test_split

In [ ]:
# 2. Definimos parámetros
waveform_db = 'mitdb'
clases_interes = ['N', 'V', 'A']  # Nos quedamos solo con las clases de interés
fs = 360  # Definimos la frecuencia de muestreo (Hz)
ventana_muestras = int(0.2 * fs)  # 0.2 segundos alrededor del pico R. * 360 Hz frecuencia = 72 muestras
mitad_ventana = ventana_muestras // 2

In [ ]:
# 3. Definimos funciones
def butter_lowpass_filter(data, cutoff=40, fs=360, order=4):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    y = filtfilt(b, a, data)
    return y

def normalizar_latido(latido):
    return (latido - np.mean(latido)) / np.std(latido)

# 4. Obtenemos lista de registros
lista_registros = wfdb.get_record_list(waveform_db)
print(f"La base de datos '{waveform_db}' contiene {len(lista_registros)} registros.")

# 5. Inicializamos lista para guardar latidos
data = []

# 6. Recorremos registros
for registro in lista_registros:
    print(f"Procesando registro: {registro}...")

    # Carga la señal y anotaciones
    record = wfdb.rdrecord(registro, pn_dir='mitdb')
    annotation = wfdb.rdann(registro, 'atr', pn_dir='mitdb')

    signal = record.p_signal[:, 0]  #Canal principal

    # Filtramos la señal completa una sola vez
    signal_filtrada = butter_lowpass_filter(signal, cutoff=40, fs=360)

    # Recorremos latidos anotados
    for index, label in zip(annotation.sample, annotation.symbol):
        if label in clases_interes:
            inicio = index - mitad_ventana
            fin = index + mitad_ventana
            if inicio >= 0 and fin <= len(signal_filtrada):
                # Segmentamos ventana de 200ms
                latido = signal_filtrada[inicio:fin]

                # Normalizamos el latido
                latido_normalizado = normalizar_latido(latido)

                # Guarda el  latido
                data.append({
                    'record': registro,
                    'sample': index,
                    'label': label,
                    'signal_normalizada': latido_normalizado
                })

# 7. Creamos el DataFrame
df = pd.DataFrame(data)

print(f"\n Procesamiento completo: {len(df)} latidos segmentados, filtrados y normalizados.")
df.head()


##2.3 División de datos
•	70% entrenamiento, 15% validación, 15% test (mezcla aleatoria).


In [ ]:

# 1. preparamos X e y
X = np.vstack(df['signal_normalizada'].values)  # Señales (cada fila = 72 muestras)
y = df['label'].values                          # Etiquetas

# 2. dividimos en entrenamiento, validación y test
from sklearn.model_selection import train_test_split

# Primera división: entrenamiento (70%) y "restante" (30%)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Segunda división: validación (15%) y test (15%)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

# Mostramos tamaños
print(f"Tamaño entrenamiento: {len(X_train)} muestras")
print(f"Tamaño validación: {len(X_val)} muestras")
print(f"Tamaño test: {len(X_test)} muestras")


#Paso 3 - Modelado

##3.1 Red Prealimentada (MLP)

###Definición de la arquitectura

In [ ]:
#Convertimos las etiquetas a enteros para que puedan ser compiladas por sparse_categorical_crossentropy
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_val = encoder.transform(y_val)
y_test = encoder.transform(y_test)

In [ ]:
#1. Importamos las librerías necesarias
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [ ]:
# 2. Definimos la arquitectura de 2 capas densas
def crear_modelo():
  model = Sequential([
      Dense(64, activation='relu', input_shape=(ventana_muestras,)),
      #Añadimos regulación dropout "apagando" el 30% de las neuronas de la capa anterior (valor intermedio entre 0.2 y 0.5 que son los valores habituales)
      #De esta forma evitamos un posible overfitting en arquitecturas con capas densas
      Dropout(0.3),
      Dense(32, activation='relu'),
      Dropout(0.3),
      Dense(3, activation='softmax')  # Probabilidad para las 3 clases: N, V, A
  ])
  return model

###Entrenamiento

1. Compararemos 2 métodos optimizadores, Adam y SDG

https://keras.io/api/optimizers/adam/

https://keras.io/api/optimizers/sgd/


In [ ]:
import time

model_adam = crear_modelo()
model_adam.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy']) #Utilizamos la función de pérdida que indica la consigna

start_adam = time.time() #Vamos a medir el tiempo de ejecución
history_adam = model_adam.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=50, #Utilizamos el máximo que indica la consigna
    batch_size=32,
    verbose=0
)
end_adam = time.time()
tiempo_adam = end_adam - start_adam

In [ ]:
from tensorflow.keras.optimizers import SGD

model_sgd = crear_modelo()
optimizer_sgd = SGD(learning_rate=0.01, momentum=0.9)
model_sgd.compile(optimizer=optimizer_sgd, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

start_sdg = time.time()
history_sgd = model_sgd.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=32,
    verbose=0
)
end_sdg = time.time()
tiempo_sdg = end_sdg - start_sdg

In [ ]:
#Precisión general
loss, acc = model_adam.evaluate(X_test, y_test)
print(f"Accuracy general Adam: {acc:.2%}")
print(f"Tiempo de entrenamiento con Adam: {tiempo_adam:.2f} segundos")

loss, acc = model_sgd.evaluate(X_test, y_test)
print(f"Accuracy general SDG: {acc:.2%}")
print(f"Tiempo de entrenamiento con Adam: {tiempo_sdg:.2f} segundos")

###Conclusión

#Paso 4 - Evaluación

##Evaluación Red Prealimentada (MLP)

In [ ]:
#Graficamos la comparación entre ambos optimizadores
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 5))

# Precisión
plt.subplot(1, 2, 1)
plt.plot(history_adam.history['accuracy'], label='Adam - Train')
plt.plot(history_adam.history['val_accuracy'], label='Adam - Val')
plt.plot(history_sgd.history['accuracy'], label='SGD - Train')
plt.plot(history_sgd.history['val_accuracy'], label='SGD - Val')
plt.title('Precisión durante el entrenamiento')
plt.xlabel('Épocas')
plt.ylabel('Accuracy')
plt.legend()

# Pérdida
plt.subplot(1, 2, 2)
plt.plot(history_adam.history['loss'], label='Adam - Train')
plt.plot(history_adam.history['val_loss'], label='Adam - Val')
plt.plot(history_sgd.history['loss'], label='SGD - Train')
plt.plot(history_sgd.history['val_loss'], label='SGD - Val')
plt.title('Pérdida durante el entrenamiento')
plt.xlabel('Épocas')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()
